In [1]:
import os
import dotenv
dotenv.load_dotenv()
groq_key = os.getenv("GROQ_API_KEY")
open_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import langchain_groq
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
llm = langchain_groq.ChatGroq(model="llama3-8b-8192")

In [3]:
doc_path = "./documents/"
docs = []
for doc in [_ for _ in os.listdir(doc_path) if str(_).endswith('.pdf')]: 
    document = PyPDFLoader(doc_path+doc)
    docs.append(document.load())
for doc in [_ for _ in os.listdir(doc_path) if str(_).endswith('.csv')]:
    document = CSVLoader(doc_path + doc)
    docs.append(document.load())

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap = 200)
splits = []
for doc in docs:
    split = text_splitter.split_documents(doc)
    splits.append(split)
splits = sum(splits,[])

In [5]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    # embedding_function = OpenAIEmbeddings(),
    )

/opt/homebrew/Caskroom/miniforge/base/envs/lang/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/homebrew/Caskroom/miniforge/base/envs/lang/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [8]:
rag_chain = (
    {
        "context": retriever | format_docs, "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
ans = rag_chain.invoke(
    "Could you tell me the average age of the students in the Student performance data file?"
    )
print(str(ans))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The average age of the students in the Student performance data file is approximately 15.5 years old, based on the provided student data.


In [10]:
import gradio as gr
def ask(prompt:str):
    return rag_chain.invoke(prompt)

demo = gr.Interface(fn = ask, inputs=["text"],outputs=["text"],) 

In [11]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
